# **Extracción del texto de los PDF**

Vamos a trabajar con documentos (.pdf) en el el cual son resoluciones judiciales de la corte suprema, y estos presentan una estructura "semi-estructurada".

## **Desafios que debemos de cumplir en esta Fase**

- **Ruido Visual y Metadatos:** Las cabeceras (CORTE SUPREMA DE JUSTICIA), los pies de página (Página X de Y) y, especialmente, los bloques de SINOE (Sistema de Notificaciones Electrónicas) son "ruido". Contienen información sobre los vocales supremos, fechas y firmas digitales que no son parte del contenido jurídico de la sentencia.
- **Diseño Multi-columna y Bloques Flotantes:** Los bloques de SINOE a menudo se presentan en columnas laterales o intercalados, rompiendo el flujo natural del texto principal. Una extracción ingenua mezclaría el contenido de estos bloques con los párrafos de la sentencia.
- **Identificadores y Títulos:** Los documentos tienen identificadores claros como "Casación Laboral N.° XXXXX-2022", que son metadatos valiosos.
- **Contenido Estructurado:** Afortunadamente, los documentos tienen una estructura lógica clara que podemos aprovechar.

## **Elementos que podemos ignorar (Ruido)**

- Todo el contenido dentro de los recuadros de SINOE.
- La cabecera repetitiva: SEGUNDA SALA DE DERECHO CONSTITUCIONAL Y SOCIAL TRANSITORIA...
- El número de página en el pie de página.
- Las iniciales de los responsables al final del documento (ej. JBA/LZCR).

## **Elementos que nos interesa seleccionar**

- **Metadatos Principales:** El número de casación (Casación Laboral N.°...), el distrito judicial (Huánuco, La Libertad), la materia (Relación laboral indeterminado y otros) y la fecha de la sentencia (uno de agosto de dos mil veinticuatro).
- **Sumilla:** Este es un resumen de oro. Es un candidato perfecto para ser un "chunk" de alta calidad o metadato por sí mismo.
- **Considerandos:** El corazón de la resolución. Están numerados (Primero., Segundo., etc.), lo que nos da un punto de anclaje perfecto para la fragmentación. Cada "Considerando" desarrolla un argumento o un punto fáctico.
- **Decisión:** La parte final (DECISIÓN:, Declararon FUNDADO...). Es la conclusión y el fallo de la corte. Debe ser capturado como una unidad separada.

Vamos a utilizar el codigo que escribimos en el script leer_pdf.py

In [5]:
# Este script probaremos la extraccion de un pdf con pymupdf
import fitz

pdf_path = "../pdfs/Resolucion_S_N_2025-01-09 15_07_49.383,id=1007485398.pdf"

try:
    doc = fitz.open(pdf_path) # Abrimos el pdf

    print(f"El PDF tiene {doc.page_count} paginas")

    full_text = ""

    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        full_text += page.get_text("text")
        full_text += "\n---Fin de la pagina---".format(page_num+1)

    doc.close()

    print("\n---Inicio del PDF---")
    print(full_text[0:500], end = "...")
    print("\n---Fin del PDF---")

except Exception as e:
    print(f"Error al abrir el PDF: {e}")

El PDF tiene 12 paginas

---Inicio del PDF---
SEGUNDA SALA DE DERECHO CONSTITUCIONAL Y SOCIAL TRANSITORIA 
CORTE SUPREMA DE JUSTICIA DE LA REPÚBLICA 
 
CASACIÓN LABORAL Nº 26240-2022   
HUÁNUCO     
Desnaturalización de contrato y otros  
PROCESO ABREVIADO LABORAL – NLPT 
 
1 
 
 
 
 
Lima, siete de noviembre de dos mil veinticuatro.- 
VISTA; la causa número veintiséis mil doscientos cuarenta, guion dos mil 
veintidós, HUÁNUCO; en audiencia pública de la fecha; y luego de efectuada la 
votación con arreglo a ley, se emite la siguiente sente...
---Fin del PDF---


Solo hemos mostrado los primeros 500 caracteres del documento, este PDF tiene 12 paginas por lo que la cantidad de palabras y caracteres que puede haber son grandes, hemos notado algunas cosas revisando estas muestras:

1. **Cabeceras SINOE:** En todos los PDF existen capturas de metadatos de SINOE pero esto tambien lo extrae pymupdf haciendo ensuciando el texto final.
2. **Metadatos de Formato:** Los números de página (1, 2, 3...), los encabezados repetitivos (SEGUNDA SALA DE DERECHO...) y los saltos de página no aportan valor semántico y deben ser eliminados.
3. **Posible Problema de Codificación:** La línea RazÃ³n: RESOLUCIÃ“N es un problema de codificación de caracteres (probablemente UTF-8 mal interpretado).
4. **Estructura Oculta:** Los patrones que buscamos: Sumilla:, CONSIDERANDO, DECISIÓN.

Vamos a realizar una extracción de bloques de texto y lo filtraremos si son ruido o no, una vez limpio recontruinos el texto "pegando" estos bloques limpios en un texto claro.

In [408]:
import re

def extraer_y_filtrar_bloques(pdf_path):
    """
    Extrae bloques de texto, y filtra semánticamente los que son ruido (SINOE, etc.)
    para reconstruir un cuerpo de texto limpio.
    """
    texto_limpio_reconstruido = ""
    
    # Palabras clave que identifican un bloque como "ruido"
    noise_keywords = [
        'SINOE', 'Vocal Supremo', 'FIRMA DIGITAL', 'Servicio Digital', 
        'Razón: RESOLUCIÓN', 'D.Judicial:', 'Secretario De Sala', 'DEPA'
    ]
    
    try:
        doc = fitz.open(pdf_path)
        for page in doc: # Recorremos cada pagina del pdf
            # Extraemos bloques ordenados por posición vertical
            bloques = page.get_text("blocks", sort=True)
            for b in bloques:
                texto_bloque = b[4] # El texto del bloque
                
                es_ruido = False
                # Filtramos el texto por palabras clave de ruido
                if any(keyword in texto_bloque for keyword in noise_keywords):
                    es_ruido = True
                
                # Filtramos si el bloque es el encabezado
                if "SEGUNDA SALA DE DERECHO CONSTITUCIONAL" in texto_bloque:
                    es_ruido = True
                    
                # Filtramos si el bloque es un pie de pagina
                if texto_bloque.strip().startswith("Página \d de \d"):
                    es_ruido = True

                # Si el bloque no es ruido lo agregamos al texto limpio
                
                if not es_ruido:
                    texto_limpio_reconstruido += texto_bloque
                    
        doc.close()
        return texto_limpio_reconstruido
    except Exception as e:
        print(f"Error en la extracción: {e}")
        return ""

def limpiar_texto(texto):
    """Normaliza espacios y saltos de línea."""
    texto = re.sub(r'\n{3,}', '\n\n', texto) # Reemplaza 3 o más saltos de línea con dos
    texto = re.sub(r' +', ' ', texto) # Reemplaza 2 o más espacios/tabs con uno
    texto = re.sub(r'\n \n', '\n', texto)
    texto = re.sub(r'\n?Página\s+\d+\s+de\s+\d+\s*\n?', '\n', texto) 
    return texto.strip() 

In [409]:
def extraer_texto(pdf_path):
    """
    Extrae el 100% del texto del PDF, sin filtros espaciales.
    """
    try:
        doc = fitz.open(pdf_path)
        texto_completo = ""
        for page in doc:
            texto_completo += page.get_text("text", sort=True) 
        doc.close()
        return texto_completo
    except Exception as e:
        print(f"Error al procesar {pdf_path}: {e}")
        return ""

In [410]:
def extraer_metadatos(texto_crudo):
    """
    Extrae metadatos estructurados de la cabecera del texto crudo.
    """
    metadatos = {}
    
    patron_casacion = r"CASACIÓN LABORAL (?:Nº|N.º)\s*(\d+-\d+)\s*([A-ZÁÉÍÓÚ\s]+?)\s+(Desnaturalización de contrato y otros|Homologación de remuneraciones y otros.*?)\s+PROCESO"
    
    match = re.search(patron_casacion, texto_crudo, re.DOTALL | re.IGNORECASE)
    
    if match:
        metadatos['n_casacion'] = match.group(1).strip()
        metadatos['lugar'] = match.group(2).strip()
        metadatos['materia'] = match.group(3).strip()
    
    return metadatos

In [411]:
def dividir_en_chunks_semanticos(texto_limpio):
    """
    La versión de chunking más simple. Toma el texto limpio y lo divide
    directamente por las secciones clave.
    """

    patron_division = r'(?=Sumilla:?|VISTA;?|MATERIA DEL RECURSO|CAUSALES DEL RECURSO|CAUSAL DEL RECURSO|CONSIDERANDO:?|DECISIÓN:?|Por estas consideraciones:\n)'
    
    chunks = re.split(patron_division, texto_limpio)
    
    chunks_filtrados = [chunk.strip() for chunk in chunks if chunk.strip()]

    while "Por estas consideraciones:" in chunks_filtrados:
        chunks_filtrados.remove("Por estas consideraciones:")
    
    return chunks_filtrados

In [417]:
texto = extraer_y_filtrar_bloques(pdf_path)

texto_limpio = limpiar_texto(texto)

texto_crudo_completo = extraer_texto(pdf_path)

metadatos = extraer_metadatos(texto_crudo_completo)

chunks = dividir_en_chunks_semanticos(texto_limpio)

documento_procesado = {
    "metadatos": metadatos,
    "chunks": chunks,
}

documento_procesado

{'metadatos': {'n_casacion': '26240-2022',
  'lugar': 'HUÁNUCO',
  'materia': 'Desnaturalización de contrato y otros'},
 'chunks': ['Sumilla: Para la procedencia de la reposición en el empleo, \nresulta exigible que el demandante haya ingresado por \nconcurso público de méritos, en una plaza vacante, \npresupuestada y de naturaleza indeterminada, conforme al \nartículo 5° de la Ley número 28175 y al criterio es tablecido \npor el Tribunal Constitucional en el Precedente Vinculante N° \n05057-2013-PA/TC JUNIN. \nPalabras clave: Reposición Laboral – Concurso Público – \nMeritocracia. \nLima, siete de noviembre de dos mil veinticuatro.-',
  'VISTA; la causa número veintiséis mil doscientos cuarenta, guion dos mil \nveintidós, HUÁNUCO; en audiencia pública de la fecha; y luego de efectuada la \nvotación con arreglo a ley, se emite la siguiente sentencia:',
  'MATERIA DEL RECURSO \nSe trata del recurso de casación interpuesto por la parte demandada, Registro \nNacional de Identificación y E